In [ ]:
!pip install geopandas
!pip install laszip
!pip install laspy
!pip install pyarrow
!pip install pydeck
!pip install rtree
!pip install scipy
!pip install tqdm

import os
import gc
import sys
import base64
import laszip
import laspy
import pandas as pd
import geopandas as gpd
import requests
from google.colab import drive
from tqdm import tqdm
from shapely.geometry import Point

DRIVE = "/content/gdrive"
ROOT = f"{DRIVE}/My Drive/Python/pix2pix"
LAZ_DIRECTORY = f"{DRIVE}/My Drive/Databases/LidarBC"

drive.mount(DRIVE)


def import_gh(file_name, repository, user="nicholasmartino", branch="master", save="imported"):
    url = f"https://raw.githubusercontent.com/{user}/{repository}/{branch}/{file_name}"
    print(url)
    if not os.path.exists(f"{save}/imported"):
        os.mkdir(f"{save}/imported")

    req = requests.get(url)

    with open(f"{save}/{file_name}", "wb") as file:
        file.write(req.content)
    return f"{save}/{file_name}"


os.chdir(ROOT)
import_gh("Fabric.py", "city")
import_gh("Network.py", "city")
import_gh("ShapeTools.py", "morphology")
sys.path.append("imported")

from imported.Fabric import Buildings
from imported.ShapeTools import Analyst


# Buildings
building_gdf = gpd.read_file(filename='tmp/ODB_BritishColumbia/odb_britishcolumbia.shp')
building_gdf.crs = 3347
building_gdf = building_gdf.to_crs(26910)
bld = Buildings(gdf=building_gdf, to_crs=26910, group_by='Build_ID') 

# Get heights from digital surface models
for laz_file in tqdm(os.listdir(LAZ_DIRECTORY)[0:2]):
    dsm = laspy.read(f"{LAZ_DIRECTORY}/{laz_file}")
    dsm_gdf = gpd.GeoDataFrame({'geometry': [Point(xyz) for xyz in zip(dsm.x, dsm.y, dsm.z)]})
    dsm_gdf['z'] = [z for z in dsm.z]
    dsm_gdf.crs = 26910
    bld.gdf = Analyst(bld.gdf, dsm_gdf).spatial_join(operations=['mean'])
    gc.collect()

bld.gdf.to_feather(f'{LAZ_DIRECTORY}/dsm_bc_02.feather')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
https://raw.githubusercontent.com/nicholasmartino/city/master/Fabric.py
https://raw.githubusercontent.com/nicholasmartino/city/master/Network.py
https://raw.githubusercontent.com/nicholasmartino/morphology/master/ShapeTools.py


  1%|          | 1/170 [02:16<6:25:02, 136.70s/it]tcmalloc: large alloc 2433802240 bytes == 0x5615f5110000 @  0x7ff678aec1e7 0x561596531008 0x5615965eb4f0 0x5615965eaf4e 0x56159653e0a4 0x5615964fec52 0x561596571c25 0x5615964ffafa 0x56159656dc0d 0x5615964ffafa 0x56159656dc0d 0x5615964ffafa 0x56159656dc0d 0x56159656c9ee 0x5615964ffbda 0x561596571d00 0x56159656c9ee 0x56159656c6f3 0x56159656ab60 0x5615964fe349 0x5615964fe240 0x561596571973 0x5615965f1dd5 0x56159656ec6f 0x5615965f1dd5 0x56159656ec6f 0x5615965f1dd5 0x5615964fe2ed 0x5615965efe1d 0x561596571e99 0x5615964ffafa
tcmalloc: large alloc 2147483648 bytes == 0x561708c10000 @  0x7ff678aee887 0x7ff6773e4c29 0x7ff6773e5afb 0x7ff6773e5bb4 0x7ff6773dcf43 0x7ff6774344ab 0x7ff677424782 0x7ff5db034dbf 0x7ff5db026a2a 0x7ff5db032c1c 0x7ff5db08463b 0x7ff5db08a540 0x7ff5db098f13 0x5615964ff098 0x5615965724d9 0x5615964ffafa 0x56159656dc0d 0x5615964ffafa 0x56159656dc0d 0x5615964ffafa 0x56159656dc0d 0x56159656c9ee 0x5615964ffbda 0x561596571d00 0x561